# Filtering Those Pesky Spam Messages
_Author: Zeth De Luna &mdash; September 5, 2020_

In this project, we're going to study the practical side of the Naive Bayes algorithm by building a spam filter for SMS messages.

To classify messages as spam or non-spam, the computer:
1. Learns how humans classify messages.
2. Uses that human knowledge to estimate probabilities for new messages &mdash; probabilities for spam and non-spam.
3. Classifies a new message based on these probability values &mdash; if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, we may need a human to classify the message).

Our first task is to "teach" the computer how to classify messages. To do this, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, which can be downloaded from [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

## Exploring the Data
Let's read in the data and get familiar with it.

In [1]:
import pandas as pd

In [2]:
spam_collection = pd.read_csv('SMSSpamCollection', sep='\t', header=None,
                              names=['Label', 'SMS'])

In [3]:
spam_collection.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
print('Number of Rows: {}'.format(spam_collection.shape[0]))
print('Number of Columns: {}'.format(spam_collection.shape[1]))

Number of Rows: 5572
Number of Columns: 2


In [5]:
spam_collection['Label'].value_counts(dropna=False)

ham     4825
spam     747
Name: Label, dtype: int64

The `SMSSpamCollection` data set has 5,572 rows (each row contains information on a single message) and has no empty rows. We see that the `Label` column can be either `ham` or `spam`. Given that this data set is about messages that were classified either as spam or non-spam, we can infer that `ham` refers to non-spam messages.

In [6]:
n_ham = spam_collection['Label'].value_counts()[0]
n_spam = spam_collection['Label'].value_counts()[1]
n_total = spam_collection.shape[0]

percent_ham = (n_ham / n_total) * 100
percent_spam = (n_spam / n_total) * 100
print('This data set is:\n{:.2f}% ham messages\n{:.2f}% spam messages'
      .format(percent_ham, percent_spam))

This data set is:
86.59% ham messages
13.41% spam messages


We can see that this data set consists mostly of non-spam messages.

## Creating a Training and Test Set
Before we actually use our spam filter, we should first test it to see if it works. So, we're going to split the data set into two categories:
* A training set, which we'll use to "train" the computer how to classify messages.
* A test set, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our data set for training, and 20% for testing, i.e.:
* The training set will have 4,458 messages (about 80% of the data).
* The test set will have 1,114 messages (about 20% of the data).

When the spam filter is ready, we're going to treat the test set as new messages and have the filter classify them. Then, we'll compare the results of the algorithm classification with the classification done by a human and see how well the filter performed. Let's set our bar at 80% accuracy &mdash; we'll conclude that the spam filter works if it can correctly classify (spam or ham) at least 80% of the messages.

First, we'll randomize our entire data set, then assign 80% of it to the training set and the remaining 20% to the test set.

In [7]:
# randomize the data
randomized = spam_collection.sample(frac=1, random_state=1)

# create training and test data sets
training_set = (randomized.iloc[:4458, :]
                    .copy()
                    .reset_index(drop=True)
               )

test_set = (randomized.iloc[4458:, :]
                .copy()
                .reset_index(drop=True)
           )

In [8]:
# check shape of both data sets
print('Training Set shape: {}'.format(training_set.shape))
print('Test Set shape: {}'.format(test_set.shape))

Training Set shape: (4458, 2)
Test Set shape: (1114, 2)


The shapes of the training set and test set look good. Let's find the percentage of spam and ham messages in both of these sets. Hopefully, they'll be representative of the original data set, `spam_collection`.

In [9]:
training_set.head(2)

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"


In [10]:
training_set['Label'].value_counts()

ham     3858
spam     600
Name: Label, dtype: int64

In [11]:
test_set['Label'].value_counts()

ham     967
spam    147
Name: Label, dtype: int64

In [12]:
# spam and ham percentages for the training set
n_ham_training = training_set['Label'].value_counts()[0]
n_spam_training = training_set['Label'].value_counts()[1]
perc_ham_training = (n_ham_training / training_set.shape[0]) * 100
perc_spam_training = (n_spam_training / training_set.shape[0]) * 100

n_ham_test = test_set['Label'].value_counts()[0]
n_spam_test = test_set['Label'].value_counts()[1]
perc_ham_test = (n_ham_test / test_set.shape[0]) * 100
perc_spam_test = (n_spam_test / test_set.shape[0]) * 100

print('Training Set:\n{:.2f}% ham messages\n{:.2f}% spam messages'
     .format(perc_ham_training, perc_spam_training))
print('')
print('Test Set:\n{:.2f}% ham messages\n{:.2f}% spam messages'
     .format(perc_ham_test, perc_spam_test))

Training Set:
86.54% ham messages
13.46% spam messages

Test Set:
86.80% ham messages
13.20% spam messages


The training and test sets both have ham-vs-spam percentages that are very similar to the original data set. These data sets would be sufficient for training and testing our spam filter.

## Cleaning the Training Data
Recall that we'll be using the Naive Bayes algorithm to calculate the probability that a message will be spam or ham. When a message is entered into the algorithm, it will make the classification based on the results it gets to the equations below:

$$ P(\textrm{Spam}|w_1,w_2,\ldots,w_n) \propto P(\textrm{Spam}) \cdot \prod_{i=1}^{n} P(w_i|\textrm{Spam}) $$

$$ P(\textrm{Ham}|w_1,w_2,\ldots, w_n) \propto P(\textrm{Ham}) \cdot \prod_{i=1}^{n} P(w_i|\textrm{Ham}) $$

Where $P(w_i|\textrm{Spam})$ and $P(w_i|\textrm{Ham})$ are given by:

$$ P(w_i|\textrm{Spam}) = \frac{N_{w_i|\textrm{Spam}} + \alpha}{N_{\textrm{Spam}} + \alpha \cdot N_{\textrm{Vocabulary}}} $$

$$ P(w_i|\textrm{Ham}) = \frac{N_{w_i|\textrm{Ham}} + \alpha}{N_{\textrm{Ham}} + \alpha \cdot N_{\textrm{Vocabulary}}} $$

Equation Terms:
* $N_{w_i|\textrm{Spam}}$ &mdash; the number of times the word $w_i$ occurs in spam messages
* $N_{w_i|\textrm{Spam}^C}$ &mdash; the number of times the word $w_i$ occurs in non-spam messages
* $N_{\textrm{Spam}}$ &mdash; total number of words in spam messages
* $N_{\textrm{Spam}^C}$ &mdash; total number of words in non-spam messages
* $N_{\textrm{Vocabulary}}$ &mdash; total number of words in the vocabulary
* $\alpha = 1$ ($\alpha$ is a smoothing parameter)

Before we can apply these equations to our data, we'll need to do some cleaning to bring the data in a format that will allow us to extract easily all the information we need. For our purposes, the easiest format would result in the table below.

|  | Label | $w_1$ | $w_2$ | $\ldots$ | $w_n$ |
|--|-------|-------|-------|----------|-------|
| 0| spam  |   2   |   4   | $\ldots$ |   1   |
| 1| ham   |   1   |   0   | $\ldots$ |   0   |
| 2| spam  |   3   |   1   | $\ldots$ |   2   |

To do this, we'll:
* Replace the `SMS` column with a series of new columns, where each column represents a unique word from the vocabulary.
* Each row describes a single message.
* All words in the vocabulary are lower case, so our comparisons should be case-insensitive.
* Punctuation is not taken into account anymore.

We'll begin by removing the punctuation from the messages and converting all letters to lower case.

In [13]:
# remove all punctuation from the SMS column in the training set
# and transform all letters to lower case
training_set['SMS'] = (training_set['SMS']
                           .str.replace(r'\W', ' ')
                           .str.lower()
                      )

In [14]:
training_set.head(2)

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan


Next, we'll create a vocabulary list containing all of the unique words present in our data. We'll compare the words in each message to this list to calculate the probability that the message is spam or not using the equations stated above.

In [15]:
# convert each message in the SMS column into a list of words
training_set['SMS'] = training_set['SMS'].str.split()

# iterate over the SMS column and create a list of unique words
vocabulary = []
for message in training_set['SMS']:
    for word in message:
        if word in vocabulary:
            pass
        else:
            vocabulary.append(word)

Now, we'll use the vocabulary list to make the data transformation described above (transform the SMS column to a series of columns).

To make the transformation, we'll first build a dictionary that contains the information on how many times each unique word appears in each message. Then, we'll create a new dataframe out of that dictionary and join it with the `training_set` dataframe.

In [16]:
# create dictionary containing zero counts for all words
word_counts_per_message = {unique_word: [0] * len(training_set['SMS'])
                           for unique_word in vocabulary}

# fill the dictionary with the proper counts per word
for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_message[word][index] += 1
        
# transform the dictionary into a dataframe
word_counts_per_message = pd.DataFrame(word_counts_per_message)

In [17]:
word_counts_per_message.head()

,yep,by,the,pretty,sculpture,yes,princess,are,you,going,...,beauty,hides,secrets,n8,jewelry,related,trade,arul,bx526,wherre
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# merge the new dataframe with the training_set dataframe
training_set_final = pd.concat([training_set, word_counts_per_message],
                               axis=1)

In [19]:
training_set_final.head()

,Label,SMS,yep,by,the,pretty,sculpture,yes,princess,are,...,beauty,hides,secrets,n8,jewelry,related,trade,arul,bx526,wherre
0,ham,"[yep, by, the, pretty, sculpture]",1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Creating the Spam Filter
Now that we're done with data cleaning and have a training set to work with, we can begin creating the spam filter. 

### Calculating the Constants
Recall the equations we'll be using for the filter:

$$ P(\textrm{Spam}|w_1,w_2,\ldots,w_n) \propto P(\textrm{Spam}) \cdot \prod_{i=1}^{n} P(w_i|\textrm{Spam}) $$

$$ P(\textrm{Ham}|w_1,w_2,\ldots, w_n) \propto P(\textrm{Ham}) \cdot \prod_{i=1}^{n} P(w_i|\textrm{Ham}) $$

Where $P(w_i|\textrm{Spam})$ and $P(w_i|\textrm{Ham})$ are given by:

$$ P(w_i|\textrm{Spam}) = \frac{N_{w_i|\textrm{Spam}} + \alpha}{N_{\textrm{Spam}} + \alpha \cdot N_{\textrm{Vocabulary}}} $$

$$ P(w_i|\textrm{Ham}) = \frac{N_{w_i|\textrm{Ham}} + \alpha}{N_{\textrm{Ham}} + \alpha \cdot N_{\textrm{Vocabulary}}} $$

The following terms are constants in these equations that we can calculate now and have at the ready:
* $P(\textrm{Spam})$ (the probability that a message is spam)
* $P(\textrm{Ham})$ (the probability that a message is not spam)
* $N_{\textrm{Spam}}$ (number of words in all of the spam messages)
* $N_{\textrm{Ham}}$ (number of words in all of the ham messages)
* $N_{\textrm{Vocabulary}}$ (number of unique words in all messages)
* Set smoothing factor to $\alpha = 1$ (Laplace Smoothing)

In [20]:
# isolate spam and ham messages
n_messages = training_set_final.shape[0]
hams = training_set_final[training_set_final['Label']=='ham']
spams = training_set_final[training_set_final['Label']=='spam']

# calculate P(Spam) and P(Ham)
p_spam = spams.shape[0] / n_messages
p_ham = hams.shape[0] / n_messages

# calculate N_spam
n_spam = spams['SMS'].apply(len).sum()
    
# calculate N_hamf
n_ham = hams['SMS'].apply(len).sum()

# calculate N_vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

### Calculating the Parameters
We've got our $P(\textrm{Spam}), P(\textrm{Ham}), N_{\textrm{Spam}}, N_{\textrm{Ham}}, N_{\textrm{Vocabulary}}$ and $\alpha$, which will remain constant regardless of the message.

Now, we'll need to calculate $P(w_i|\textrm{Spam})$ and $P(w_i|\textrm{Ham})$, which vary depending on the individual words. We can use our training set to calculate the probabilities, $P(w_i|\textrm{Spam})$ and $P(w_i|\textrm{Ham})$, for each word in our vocabulary. These probability values are called parameters.

We'll use equations below to calculate the probabilities for each word:

$$ P(w_i|\textrm{Spam}) = \frac{N_{w_i|\textrm{Spam}} + \alpha}{N_{\textrm{Spam}} + \alpha \cdot N_{\textrm{Vocabulary}}} $$

$$ P(w_i|\textrm{Ham}) = \frac{N_{w_i|\textrm{Ham}} + \alpha}{N_{\textrm{Ham}} + \alpha \cdot N_{\textrm{Vocabulary}}} $$

First, we'll initialize two dictionaries, where each key-value pair is a unique word (from our vocabulary) represented as a string, and the value is 0. One dictionary will store the parameters for $P(w_i|\textrm{Spam})$ and the other will store the parameteres for $P(w_i|\textrm{Ham})$.

In [21]:
# initialize dictionaries for P(w_i|Spam) and P(w_i|Ham)
param_spam = {unique_word: 0 for unique_word in vocabulary}
param_ham = {unique_word: 0 for unique_word in vocabulary}

# calculate parameters
for word in vocabulary:
    # get N_(w_i|Spam) and N_(w_i|Ham) for the current word
    n_wispam = spams[word].sum()
    n_wiham = hams[word].sum()
    # calculate P(w_i|Spam) and P(w_i|Ham)
    p_word_spam = (n_wispam + alpha) / ((n_spam + alpha) * n_vocabulary)
    p_word_ham = (n_wiham + alpha) / ((n_ham + alpha) * n_vocabulary)
    # assign value to key in dictionaries
    param_spam[word] = p_word_spam
    param_ham[word] = p_word_ham

### Building the Spam Filter
Now that we've calculated all the constants and parameters we need, we can finally start creating the spam filter. The spam filter can be understood as a function that:
* Takes in as input a new message ($w_1, w_2, \ldots, w_n$)
* Calculates $P(\textrm{Spam}|w_1, w_2, \ldots, w_n)$ and $P(\textrm{Ham}|w_1, w_2, \ldots, w_n)$
* Compares the values of $P(\textrm{Spam}|w_1, w_2, \ldots, w_n)$ and $P(\textrm{Ham}|w_1, w_2, \ldots, w_n)$ and:
    * If $P(\textrm{Ham}|w_1, w_2, \ldots, w_n)$ > $P(\textrm{Spam}|w_1, w_2, \ldots, w_n)$, then the message is classified as ham.
    * If $P(\textrm{Ham}|w_1, w_2, \ldots, w_n)$ < $P(\textrm{Spam}|w_1, w_2, \ldots, w_n)$, then the message is classified as spam.
    * If $P(\textrm{Ham}|w_1, w_2, \ldots, w_n)$ = $P(\textrm{Spam}|w_1, w_2, \ldots, w_n)$, then the algorithm may request human help.
    
Below, we'll create this function.

In [22]:
import re

def classify(message):
    '''This function will classify an input message as either
    spam or not spam.'''
    # clean the message and transform it into a list of words
    message = re.sub(r'\W', ' ', message)
    message = message.lower().split()

    # initiate probabilities
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # iterate over each word in the message and calculate probability value
    for word in message:
        if word in param_spam:
            p_spam_given_message *= param_spam[word]
        if word in param_ham:
            p_ham_given_message *= param_ham[word]
    
    print('P(Spam|message): {}'.format(p_spam_given_message))
    print('P(Ham|message): {}'.format(p_ham_given_message))
    
    # classify the message
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal probabilities, have a human classify this :(')

Let's test this function with one spam message and one ham message.

In [23]:
message_spam = 'WINNER!! This is the secret code to unlock the money: C3421'
message_ham = 'Sounds good, Tom, then see u there'

print('Spam Message Test:')
classify(message_spam)

print('\nHam Message Test:')
classify(message_ham)

Spam Message Test:
P(Spam|message): 5.32218937227696e-59
P(Ham|message): 5.821370428319695e-62
Label: Spam

Ham Message Test:
P(Spam|message): 2.5485202709195435e-51
P(Ham|message): 5.2028823344528615e-48
Label: Ham


Our classification function seems to work.

## Measuring the Accuracy of the Spam Filter
Now that we have a working spam filter, we'll apply it to our test set and measure its accuracy using

$$ \textrm{Accuracy} = \frac{\textrm{number of correctly classified messages}}{\textrm{total number of classified messages}} $$

To make things easier, we'll create a new column in `test_set`, named `predicted`, that will show the results of our classification function for each message in the data set. Then, we'll count the number of correctly classified messages and apply it to the equation above.

To apply our function to the test set, we'll need to alter the function so that it returns the labels instead of printing them.

In [24]:
def classify_test_set(message):
    '''This function will classify an input message as either
    spam or not spam.'''
    # clean the message and transform it into a list of words
    message = re.sub(r'\W', ' ', message)
    message = message.lower().split()

    # initiate probabilities
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # iterate over each word in the message and calculate probability value
    for word in message:
        if word in param_spam:
            p_spam_given_message *= param_spam[word]
        if word in param_ham:
            p_ham_given_message *= param_ham[word]
    
    # classify the message
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'need human classification'

Apply the function to the test set and create the new column.

In [25]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)

In [26]:
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now, we'll count the number of correctly classified messages and calculate our function's accuracy.

In [27]:
correct = 0
total = test_set.shape[0]

# count correct classifications
for index, row in test_set.iterrows():
    if row[0] == row[2]:
        correct += 1
        
# calculate the accuracy
accuracy = correct / total

print('Correct: {}'.format(correct))
print('Incorrect: {}'.format(total - correct))
print('Accuracy: {}'.format(accuracy))

Correct: 1078
Incorrect: 36
Accuracy: 0.9676840215439856


The classification function that we created is accurate about 96.8% of the time. Out of 1,114 new messages, our filter was able to correctly classify 1,078 of them.

## Conclusions
Recall that our goal for this project was to create a spam filter using the multinomial Naive Bayes algorithm with at least 80% accuracy. We managed to create a spam filter with an accuracy of 96.77% on the test set, which greatly exceeds our initial standard of 80% accuracy. To further improve our spam filter, we can:
* Investigate the 36 incorrectly classified messages and figure out why the algorithm reached the wrong conclusions.
* Make the filtering process more complex by making the algorithm case-sensitive.